<a href="https://colab.research.google.com/github/Anirudh-R-1201/Football_xG/blob/master/PLfantasyScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install selenium
import pandas as pd
import re
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime

     |████████████████████████████████| 911kB 5.7MB/s 


In [4]:
injuries_url = 'https://www.fantasyfootballscout.co.uk/fantasy-football-injuries/'
injury_tables = pd.read_html(injuries_url, encoding='utf-8')
injuries = injury_tables[0]
injuries.head(10)

,Name,Club,Status,Return Date,Latest News,Last Updated
0,Guendouzi (Matteo),ARS,On Loan,01/07/2021,Joined Hertha Berlin on loan until the end of ...,05/10/2020
1,Kolasinac (Sead),ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,10/12/2020
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,11/12/2020
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,09/12/2020
4,Mavropanos (Konstantinos),ARS,On Loan,01/07/2021,Joined Stuttgart on loan until the end of the ...,10/08/2020
5,Nelson (Reiss),ARS,Doubt 75%,13/12/2020,Head injury Reportedly missed the Europa Leagu...,10/12/2020
6,Ozil (Mesut),ARS,Unavailable,Unknown,Not included in Arsenal's 25-man Premier Leagu...,20/10/2020
7,Sokratis,ARS,Unavailable,Unknown,Not included in Arsenal's 25-man Premier Leagu...,21/10/2020
8,Partey (Thomas),ARS,Injured,Unknown,Thigh injury Substituted in the match against ...,09/12/2020
9,Pepe (Nicolas),ARS,Suspended,16/12/2020,3 Match Ban Suspended for three matches after ...,02/12/2020


In [13]:
injuries['last_name'] = injuries['Name'].str.split('(').str.get(0)
injuries['first_name'] = injuries['Name'].str.split('(').str.get(1).str.strip(')')
#if injuries['first_name'].bool() :
injuries['full_name'] = injuries['first_name'] + ' ' + injuries['last_name'] # | injuries['last_name']
#else :
#  injuries['full_name'] = injuries['last_name']
injuries.head()

,Name,Club,Status,Return Date,Latest News,Last Updated,last_name,first_name,full_name,first_name_regex,last_name_regex,full_name_regex
0,Guendouzi (Matteo),ARS,On Loan,01/07/2021,Joined Hertha Berlin on loan until the end of ...,05/10/2020,Guendouzi,Matteo,Matteo Guendouzi,Matteo,Guendouzi,Matteo Guendouzi
1,Kolasinac (Sead),ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,10/12/2020,Kolasinac,Sead,Sead Kolasinac,Sead,Kolasinac,Sead Kolasinac
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,11/12/2020,Luiz,NaN,NaN,NaN,NaN,NaN
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,09/12/2020,Gabriel Martinelli,NaN,NaN,NaN,NaN,NaN
4,Mavropanos (Konstantinos),ARS,On Loan,01/07/2021,Joined Stuttgart on loan until the end of the ...,10/08/2020,Mavropanos,Konstantinos,Konstantinos Mavropanos,Konstantinos,Mavropanos,Konstantinos Mavropanos


In [14]:
injuries['first_name_regex'] = injuries['Name'].str.extract(r'\(([\w-]+)\)', expand=True)
injuries['last_name_regex'] = injuries['Name'].str.extract(r'^(.+?)\s\(', expand=True)
injuries['full_name_regex'] = injuries['first_name_regex'] + ' ' + injuries['last_name_regex']

injuries.head()

,Name,Club,Status,Return Date,Latest News,Last Updated,last_name,first_name,full_name,first_name_regex,last_name_regex,full_name_regex
0,Guendouzi (Matteo),ARS,On Loan,01/07/2021,Joined Hertha Berlin on loan until the end of ...,05/10/2020,Guendouzi,Matteo,Matteo Guendouzi,Matteo,Guendouzi,Matteo Guendouzi
1,Kolasinac (Sead),ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,10/12/2020,Kolasinac,Sead,Sead Kolasinac,Sead,Kolasinac,Sead Kolasinac
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,11/12/2020,Luiz,NaN,NaN,NaN,NaN,NaN
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,09/12/2020,Gabriel Martinelli,NaN,NaN,NaN,NaN,NaN
4,Mavropanos (Konstantinos),ARS,On Loan,01/07/2021,Joined Stuttgart on loan until the end of the ...,10/08/2020,Mavropanos,Konstantinos,Konstantinos Mavropanos,Konstantinos,Mavropanos,Konstantinos Mavropanos


In [15]:
injuries[injuries['full_name'].isna()]

,Name,Club,Status,Return Date,Latest News,Last Updated,last_name,first_name,full_name,first_name_regex,last_name_regex,full_name_regex
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,11/12/2020,Luiz,NaN,NaN,NaN,NaN,NaN
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,09/12/2020,Gabriel Martinelli,NaN,NaN,NaN,NaN,NaN
7,Sokratis,ARS,Unavailable,Unknown,Not included in Arsenal's 25-man Premier Leagu...,21/10/2020,Sokratis,NaN,NaN,NaN,NaN,NaN
16,Wesley,AVL,Injured,Unknown,Knee injury Taken off with a knee injury again...,11/12/2020,Wesley,NaN,NaN,NaN,NaN,NaN
26,Alexis Mac Allister,BHA,Doubt 75%,13/12/2020,Lack of match fitness Back in training and wor...,10/12/2020,Alexis Mac Allister,NaN,NaN,NaN,NaN,NaN
32,Gudmundsson,BUR,Doubt 25%,Unknown,Hamstring injury Didn't feature against Everto...,11/12/2020,Gudmundsson,NaN,NaN,NaN,NaN,NaN
60,Steven Sessegnon,FUL,On Loan,01/07/2021,Joined Bristol City on loan until the end of t...,08/09/2020,Steven Sessegnon,NaN,NaN,NaN,NaN,NaN
67,Diego Llorente,LEE,Injured,29/12/2020,Muscle injury Ruled out for up to three weeks ...,09/12/2020,Diego Llorente,NaN,NaN,NaN,NaN,NaN
70,Ricardo Pereira,LEI,Injured,26/12/2020,"Groin Injury Suffered a ""slight"" groin strain ...",11/12/2020,Ricardo Pereira,NaN,NaN,NaN,NaN,NaN
74,Adrien Silva,LEI,Unavailable,Unknown,Joined Sampdoria on a permanent deal on 3/10. ...,07/10/2020,Adrien Silva,NaN,NaN,NaN,NaN,NaN


In [16]:
injuries['last_name'] = injuries['last_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
injuries['full_name'] = injuries['full_name'].fillna(injuries['last_name'])

print(injuries['full_name'][injuries['first_name'].isna()])
print('')

injuries = injuries[['full_name', 'Club', 'Status', 'Return Date', 'Latest News', 'Last Updated']]
injuries.columns = injuries.columns.str.lower().str.replace(' ', '_')
injuries.head()

2                     Luiz
3       Gabriel Martinelli
7                 Sokratis
16                  Wesley
26     Alexis Mac Allister
32             Gudmundsson
60        Steven Sessegnon
67          Diego Llorente
70         Ricardo Pereira
74            Adrien Silva
76        Thiago Alcantara
81                 Alisson
83              Diogo Jota
86                H Wilson
88                Angelino
94             Diogo Dalot
122              Josh Sims
129         Ryan Sessegnon
131        Matheus Pereira
133                 Hegazi
140            Xande Silva
144        Felipe Anderson
147              R Bennett
148           Leo Bonatini
149                  Jonny
150           Bruno Jordao
154           Raul Jimenez
156          Ruben Vinagre
Name: full_name, dtype: object



,full_name,club,status,return_date,latest_news,last_updated
0,Matteo Guendouzi,ARS,On Loan,01/07/2021,Joined Hertha Berlin on loan until the end of ...,05/10/2020
1,Sead Kolasinac,ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,10/12/2020
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,11/12/2020
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,09/12/2020
4,Konstantinos Mavropanos,ARS,On Loan,01/07/2021,Joined Stuttgart on loan until the end of the ...,10/08/2020


In [17]:
injuries['last_updated'] = pd.to_datetime(injuries['last_updated'], format='%d/%m/%Y')
injuries.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,full_name,club,status,return_date,latest_news,last_updated
0,Matteo Guendouzi,ARS,On Loan,01/07/2021,Joined Hertha Berlin on loan until the end of ...,2020-10-05
1,Sead Kolasinac,ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,2020-12-10
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,2020-12-11
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,2020-12-09
4,Konstantinos Mavropanos,ARS,On Loan,01/07/2021,Joined Stuttgart on loan until the end of the ...,2020-08-10


In [19]:
#today = datetime.date.today()
one_week_ago = pd.Timestamp('now').floor('D') + pd.offsets.Day(-7)
recent_injuries = injuries[injuries['last_updated'] >= one_week_ago]
recent_injuries.head()

,full_name,club,status,return_date,latest_news,last_updated
1,Sead Kolasinac,ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,2020-12-10
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,2020-12-11
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,2020-12-09
5,Reiss Nelson,ARS,Doubt 75%,13/12/2020,Head injury Reportedly missed the Europa Leagu...,2020-12-10
8,Thomas Partey,ARS,Injured,Unknown,Thigh injury Substituted in the match against ...,2020-12-09


In [21]:
next_fixtures = datetime.datetime(2018, 10, 23)
unknown_return = injuries[injuries['return_date'] == 'Unknown']

return_too_late = injuries[injuries['return_date'] != 'Unknown']
return_too_late['return_date'] = pd.to_datetime(return_too_late['return_date'], format='%d/%m/%Y')
return_too_late = return_too_late[return_too_late['return_date'] >= next_fixtures]

misses_next_match = unknown_return.append(return_too_late).sort_values(by=['last_updated'], ascending=False)
misses_next_match.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,full_name,club,status,return_date,latest_news,last_updated
84,Konstantinos Tsimikas,LIV,Doubt 75%,2020-12-13 00:00:00,Knock Substituted with a knock in the Champion...,2020-12-11
80,Alex Oxlade-Chamberlain,LIV,Doubt 75%,2020-12-13 00:00:00,Lack of match fitness Recovered from a knee in...,2020-12-11
70,Ricardo Pereira,LEI,Injured,2020-12-26 00:00:00,"Groin Injury Suffered a ""slight"" groin strain ...",2020-12-11
72,Timothy Castagne,LEI,Doubt 25%,2020-12-20 00:00:00,Hamstring injury Was in contention to face Liv...,2020-12-11
75,Caglar Söyüncü,LEI,Doubt 25%,2020-12-20 00:00:00,Groin Injury Substituted with a recurrence of ...,2020-12-11


In [22]:
doubtful = injuries[injuries['status'].str.contains('Doubt')]
doubtful.head()

,full_name,club,status,return_date,latest_news,last_updated
1,Sead Kolasinac,ARS,Doubt 75%,13/12/2020,Knock Reportedly missed the Europa League tie ...,2020-12-10
2,Luiz,ARS,Doubt 75%,13/12/2020,Head injury Suffered a cut to the head in the ...,2020-12-11
3,Gabriel Martinelli,ARS,Doubt 25%,26/12/2020,Lack of match fitness Recovered from a knee in...,2020-12-09
5,Reiss Nelson,ARS,Doubt 75%,13/12/2020,Head injury Reportedly missed the Europa Leagu...,2020-12-10
13,Keinan Davis,AVL,Doubt 50%,12/12/2020,Ankle injury Missed the Brighton game on 21/11...,2020-12-11


In [23]:
xg_url = 'https://understat.com/league/EPL'
xg_data = requests.get(xg_url)
xg_html = xg_data.content
soup = BeautifulSoup(xg_html, 'html.parser')

In [24]:
print(soup.title)

<title>EPL xG Table and Scorers for the 2020/2021 season | Understat.com</title>


In [31]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 14 not upgraded.
Need to get 81.0 MB of archives.
After this operation, 273 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 87.0.4280.66-0ubuntu0.18.04.1 [1,122 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 87.0.4280.66-0ubuntu0.18.04.1 [71.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 87.0.428

In [33]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=options)
driver.get(xg_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [34]:
headers = soup.find('div', attrs={'class':'chemp margin-top jTable'}).find('table').find_all('th',attrs={'class':'sort'})
headers

[<th class="sort" data-num="0"><span>№</span></th>,
 <th class="sort" data-num="1"><span title="Team name">Team</span></th>,
 <th class="sort" data-num="2"><span title="Matches">M</span></th>,
 <th class="sort" data-num="3"><span title="Wins">W</span></th>,
 <th class="sort" data-num="4"><span title="Draws">D</span></th>,
 <th class="sort" data-num="5"><span title="Loses">L</span></th>,
 <th class="sort" data-num="6"><span title="Goals for">G</span></th>,
 <th class="sort" data-num="7"><span title="Goals againist">GA</span></th>,
 <th class="sort" data-num="8"><span title="Points">PTS</span></th>,
 <th class="sort" data-num="9"><span class="title-expected" title="Expected Goals for">xG</span></th>,
 <th class="sort" data-num="11"><span class="title-expected" title="Expected Goals againist">xGA</span></th>,
 <th class="sort" data-num="18"><span class="title-expected" title="Expected Points">xPTS</span></th>]

In [35]:
headers_list = []
for header in headers:
    headers_list.append(header.get_text(strip=True))
print(headers_list)

['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xGA', 'xPTS']


In [36]:
body = soup.find('div', attrs={'class':'chemp margin-top jTable'}).table.tbody
all_rows_list = []
for tr in body.find_all('tr'):
    row = tr.find_all('td')
    current_row = []
    for item in row:
        current_row.append(item.get_text(strip=True))   
    all_rows_list.append(current_row)

xg_df = pd.DataFrame(all_rows_list, columns=headers_list)
xg_df

,№,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS
0,1,Tottenham,11,7,3,1,23,9,24,17.39-5.61,12.17+3.17,17.91-6.09
1,2,Liverpool,11,7,3,1,26,17,24,23.10-2.90,14.69-2.31,21.12-2.88
2,3,Chelsea,11,6,4,1,25,11,22,21.05-3.95,9.56-1.44,21.83-0.17
3,4,Leicester,11,7,0,4,21,15,21,18.46-2.54,15.40+0.40,17.52-3.48
4,5,West Ham,12,6,2,4,20,15,20,18.47-1.53,16.35+1.35,17.15-2.85
5,6,Southampton,11,6,2,3,21,17,20,12.27-8.73,15.17-1.83,13.56-6.44
6,7,Manchester United,10,6,1,3,19,17,19,15.10-3.90,14.12-2.88,14.78-4.22
7,8,Manchester City,10,5,3,2,17,11,18,17.93+0.93,10.61-0.39,19.02+1.02
8,9,Everton,11,5,2,4,20,18,17,19.11-0.89,15.92-2.08,16.57-0.43
9,10,Wolverhampton Wanderers,11,5,2,4,11,15,17,10.86-0.14,15.04+0.04,12.64-4.36


In [37]:
from google.colab import files
xg_df.to_csv('xg_df.csv') 
files.download('xg_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>